In [1]:
#read the data into a pandas dataframe
import pandas as pd
import numpy as np
import csv

raw_data = pd.read_csv("Data_Project_Data.csv")
df = pd.DataFrame(raw_data)
df

,pitch_id,pitcher_id,pitcher_side,pitch_type,pitch_initial_speed_a,break_x_a,break_z_a,pitch_initial_speed_b,spinrate_b,break_x_b,break_z_b
0,1,15865,R,Fastball,89.7,-1.23,10.06,90.7214,2302.52,4.29402,-12.2448
1,2,15865,R,Fastball,89.9,-1.81,12.27,90.8140,2262.55,3.74221,-12.3771
2,3,15865,R,Fastball,91.9,-1.84,10.62,92.6839,2211.62,4.06159,-16.3200
3,4,15865,R,Fastball,90.7,-5.82,6.26,91.5610,2269.47,10.23910,-20.5354
4,5,15865,R,Fastball,88.9,-0.69,10.04,90.0486,2229.72,2.44025,-12.6229
...,...,...,...,...,...,...,...,...,...,...,...
4995,496,495134,R,NaN,NaN,NaN,NaN,85.8881,1844.23,13.57330,-45.2486
4996,497,495134,R,NaN,NaN,NaN,NaN,78.8214,2106.01,8.31108,-56.0769
4997,498,495134,R,NaN,NaN,NaN,NaN,79.0890,2009.52,8.05088,-56.4618
4998,499,495134,R,NaN,NaN,NaN,NaN,84.8703,1838.71,15.25150,-46.2941


In [2]:
df = df.set_index('pitch_id')

In [3]:
#change pitch_type label from string to a float so it can be passed into the RF ML algorithm 
# make FB = 1, CV = 2, SL = 3, CH = 4
df['pitch_type'] = df['pitch_type'].map({'Fastball': 1, 'Curveball': 2, 'Slider': 3, 'Changeup': 4})
df['pitcher_side'] = df['pitcher_side'].map({'R': 1, 'L': 0})
#df

In [4]:
#split the data into 2 dataframes, one with machine A and the other with machine B
df_filter_for_A = pd.isnull(df["pitch_initial_speed_b"]) 
df_A = df[df_filter_for_A]
#df_A

In [5]:
df_A.drop(['pitch_initial_speed_b', 'spinrate_b', 'break_x_b', 'break_z_b'], axis=1, inplace = True)

/Users/brendanjenkins/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
#df_A

In [7]:
#filter for B
df_filter_for_B = pd.notna(df["pitch_initial_speed_b"]) 
df_B = df[df_filter_for_B]
#df_B

In [8]:
#drop A columns
df_B.drop(['pitch_initial_speed_a', 'break_x_a', 'break_z_a'], axis=1, inplace = True)

In [9]:
#df_B

In [10]:
#forward fill missing data
df_B["spinrate_b"].fillna(method = 'ffill', inplace = True)
df_B["break_x_b"].fillna(method = 'ffill', inplace = True) 
df_B["break_z_b"].fillna(method = 'ffill', inplace = True) 

/Users/brendanjenkins/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [11]:
#split Dataframes so that our test class with no pitch type are isolated
df_train_A = df_A[df_A['pitch_type'].notna()]
df_train_B = df_B[df_B['pitch_type'].notna()]

In [12]:
#dffff = df_train_A[df_train_A.isna().any(axis=1)]
#dffff

In [13]:
#drop the rows where there is a pitch_type classification but no data
df_train_A = df_train_A.dropna()

In [14]:
#drop test rows from dataframes to train RF
df_test_A = df_A[df_A['pitch_type'].isnull()]
df_test_B = df_B[df_B['pitch_type'].isnull()]

df_test_A = df_test_A.drop(columns=['pitch_type'])
df_test_B= df_test_B.drop(columns=['pitch_type'])


In [15]:
#split df_train_A and df_train_B into features and labels for testing 
df_A_y = df_train_A['pitch_type']
df_B_y = df_train_B['pitch_type']
df_A_X = df_train_A.drop(columns=['pitch_type'])
df_B_X = df_train_B.drop(columns=['pitch_type'])


In [16]:
df_test_A = df_test_A.dropna()

In [17]:
#df_A_X

In [18]:
#df_B_X

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [20]:
#train dataframe A
X_train, X_test, y_train, y_test = train_test_split(df_A_X, df_A_y, random_state = 0)

clf = RandomForestClassifier(random_state = 0)
#clf = GradientBoostingClassifier(learning_rate = .5, random_state = 0)
clf.fit(X_train, y_train)

print('Pitch Dataset for A')
print('Accuracy of RF classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Pitch Dataset for A
Accuracy of RF classifier on training set: 1.00
Accuracy of RF classifier on test set: 0.98


/Users/brendanjenkins/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [21]:
#predict Unclassified A
A_pred = clf.predict(df_test_A)

In [22]:
#insert the predicted column back into the dataframe
df_test_A.insert(loc=2, column='pitch_type', value=A_pred)

In [23]:
df_test_A

,pitcher_id,pitcher_side,pitch_type,pitch_initial_speed_a,break_x_a,break_z_a
pitch_id,,,,,,
401,15865,1,3.0,84.9,0.60,4.18
402,15865,1,3.0,85.0,1.16,1.22
403,15865,1,2.0,75.0,4.62,-4.65
404,15865,1,3.0,81.0,2.75,-1.93
405,15865,1,3.0,82.9,0.25,-1.47
...,...,...,...,...,...,...
446,495134,1,1.0,84.5,-8.06,-7.46
447,495134,1,1.0,83.9,-4.58,-9.49
448,495134,1,1.0,84.7,-5.10,-8.87


In [24]:
#train dataframe B
X_train, X_test, y_train, y_test = train_test_split(df_B_X, df_B_y, random_state = 0)

clf = RandomForestClassifier(random_state = 0)
#clf = GradientBoostingClassifier(learning_rate = .5, random_state = 0)
clf.fit(X_train, y_train)

print('Pitch Dataset for B')
print('Accuracy of RF classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Pitch Dataset for B
Accuracy of RF classifier on training set: 1.00
Accuracy of RF classifier on test set: 0.98


/Users/brendanjenkins/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
#predict Unclassified B

B_pred = clf.predict(df_test_B)

In [26]:
df_test_B

,pitcher_id,pitcher_side,pitch_initial_speed_b,spinrate_b,break_x_b,break_z_b
pitch_id,,,,,,
451,15865,1,92.5130,2254.16,16.86280,-18.6579
452,15865,1,85.4431,2254.16,-3.31916,-36.2185
453,15865,1,93.1354,2307.84,8.22619,-19.8200
454,15865,1,92.1324,2257.92,11.17950,-20.5573
455,15865,1,93.0617,2327.81,15.13490,-20.2942
...,...,...,...,...,...,...
496,495134,1,85.8881,1844.23,13.57330,-45.2486
497,495134,1,78.8214,2106.01,8.31108,-56.0769
498,495134,1,79.0890,2009.52,8.05088,-56.4618


In [27]:
df_test_B.insert(loc=2, column='pitch_type', value=B_pred)

In [28]:
df_test_B

,pitcher_id,pitcher_side,pitch_type,pitch_initial_speed_b,spinrate_b,break_x_b,break_z_b
pitch_id,,,,,,,
451,15865,1,1.0,92.5130,2254.16,16.86280,-18.6579
452,15865,1,3.0,85.4431,2254.16,-3.31916,-36.2185
453,15865,1,1.0,93.1354,2307.84,8.22619,-19.8200
454,15865,1,1.0,92.1324,2257.92,11.17950,-20.5573
455,15865,1,1.0,93.0617,2327.81,15.13490,-20.2942
...,...,...,...,...,...,...,...
496,495134,1,1.0,85.8881,1844.23,13.57330,-45.2486
497,495134,1,4.0,78.8214,2106.01,8.31108,-56.0769
498,495134,1,4.0,79.0890,2009.52,8.05088,-56.4618


In [29]:
frame = [df_test_A, df_test_B]
result = pd.concat(frame)

/Users/brendanjenkins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [30]:
result
#pitch_id pitcher_id pitcher_side pitch_type	pitch_initial_speed_a	break_x_a	break_z_a	pitch_initial_speed_b	spinrate_b	break_x_b	break_z_b
final_result = result[['pitcher_id', 'pitcher_side', 'pitch_type', 'pitch_initial_speed_a', 'break_x_a', 'break_z_a','pitch_initial_speed_b','spinrate_b','break_x_b', 'break_z_b']]

In [31]:
#final_result

In [32]:
final_result['pitch_type'] = final_result['pitch_type'].map({1:'Fastball', 2:'Curveball', 3: 'Slider', 4: 'Changeup'})
final_result['pitcher_side'] = final_result['pitcher_side'].map({1 :'R', 0: 'L'})

In [33]:
final_result

,pitcher_id,pitcher_side,pitch_type,pitch_initial_speed_a,break_x_a,break_z_a,pitch_initial_speed_b,spinrate_b,break_x_b,break_z_b
pitch_id,,,,,,,,,,
401,15865,R,Slider,84.9,0.60,4.18,NaN,NaN,NaN,NaN
402,15865,R,Slider,85.0,1.16,1.22,NaN,NaN,NaN,NaN
403,15865,R,Curveball,75.0,4.62,-4.65,NaN,NaN,NaN,NaN
404,15865,R,Slider,81.0,2.75,-1.93,NaN,NaN,NaN,NaN
405,15865,R,Slider,82.9,0.25,-1.47,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
496,495134,R,Fastball,NaN,NaN,NaN,85.8881,1844.23,13.57330,-45.2486
497,495134,R,Changeup,NaN,NaN,NaN,78.8214,2106.01,8.31108,-56.0769
498,495134,R,Changeup,NaN,NaN,NaN,79.0890,2009.52,8.05088,-56.4618


In [36]:
final_result = final_result.reset_index()

In [37]:
final_result.to_csv (r'pitch_classifications.csv', index = False, header=True)